In [2]:
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
from src.inception_cifar100 import inceptionv3
from src.temperature_scaling import ModelWithTemperature
import numpy as np
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dict_path = "C:\\Users\\jiayang\\ipynb\\trainedModel\\Inception_CIFAR100.pth"
model = inceptionv3()
model.load_state_dict(torch.load(dict_path, map_location=device, weights_only=True))
model.to(device)

# preprocess the images from CIFAR10
data_transform = transforms.Compose([
    transforms.ToTensor(),         
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  
])

# load data set from CIFAR100
dataset = CIFAR100(root="../../data", train=False, download=True,transform=data_transform)

# Temperature Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=0.5).to(device)
model.set_temperature(temp_scal_loader)
model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.394, ECE: 0.126
Optimal temperature: 0.633
After temperature - NLL: 1.890, ECE: 0.208
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.9967255115509033
Total set size: 17355
Total coverage sets: 4535
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 3.471
Average Coverage Rate in runs 1: 0.907

Running experiment 2/10...
q_hat = 0.9962914645671844
Total set size: 17108
Total coverage sets: 4495
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 3.4216
Average Coverage Rate in runs 2: 0.899

Running experiment 3/10...
q_hat = 0.9962484061717987
Total set size: 17061
Total coverage sets: 4484
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 3.4122
Average Coverage Rate in runs 3: 0.8968

Running experiment 4/10...
q_hat = 0.9958017051219941
Total set size: 16228
Total coverage sets: 4461
Total samples amount: 5000
Average Predicti

#  Result

From the above test, following results can be collected :
- Final Average Prediction Set Size: 3.44/100
- Final Average Coverage: 90.03% ($\alpha$=0.1)